# BibRec

### RF Features: Country

In [1]:
from bibrec.server.Utils import get_books, get_users, get_ratings

books = get_books("data/BX-Books.csv")
users = get_users("data/BX-Users.csv")
ratings = get_ratings("data/BX-Book-Ratings.csv")

/run/media/fab/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(path, sep=";", encoding="latin-1")
/run/media/fab/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:24: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [3]:
explicit_ratings = ratings[ratings.book_rating != 0]

# merge datasets
#df =  ratings.merge(books[['isbn', 'book_title']])
df =  explicit_ratings.merge(users[['user_id', 'age', 'country', 'state', 'city']])
df.head()

,user_id,isbn,book_rating,age,country,state,city
0,276726,0155061224,5,NaN,usa,washington,seattle
1,276729,052165615X,3,16.0,croatia,n/a,rijeka
2,276729,0521795028,6,16.0,croatia,n/a,rijeka
3,276736,3257224281,8,NaN,austria,salzburg,salzburg
4,276737,0600570967,6,14.0,australia,new south wales,sydney


In [4]:
country = pd.get_dummies(df['country'],drop_first=True)

In [5]:
df = pd.concat([df, country], axis = 1)
df.head()

,user_id,isbn,book_rating,age,country,state,city,",",", portugal","\""n/a\""""",...,", mexico",", netherlands",", new zealand",", portugal",", the netherlands",", united kingdom","block-o,r.k.puram,sector-13,new delhi , india, india","emilia, italy","quebec, canada","usa, rhode island, usa"
0,276726,0155061224,5,NaN,usa,washington,seattle,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,276729,052165615X,3,16.0,croatia,n/a,rijeka,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,276729,0521795028,6,16.0,croatia,n/a,rijeka,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,276736,3257224281,8,NaN,austria,salzburg,salzburg,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,276737,0600570967,6,14.0,australia,new south wales,sydney,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# remove duplicate columns
df = df.loc[:,~df.columns.duplicated()]

In [7]:
test = df.drop(['isbn', 'user_id', 'age', 'country', 'state', 'city'], axis = 1)
test

,book_rating,",",", portugal","\""n/a\""""","\n/a\""""","\n/a\"", greece""","aberdeenshire, united kingdom",afghanistan,"alabama, usa",alachua,...,", mexico",", netherlands",", new zealand",", portugal",", the netherlands",", united kingdom","block-o,r.k.puram,sector-13,new delhi , india, india","emilia, italy","quebec, canada","usa, rhode island, usa"
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433666,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
433667,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
433668,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
433669,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X = test.drop('book_rating', axis = 1) #Features
Y = test['book_rating']   # Target Variables

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

In [10]:
rfc = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf= 0)
rfc.fit(X_train, y_train)

RandomForestClassifier(min_weight_fraction_leaf=0)

In [11]:
rfc_pred = rfc.predict(X_test)

In [12]:
print(classification_report(y_test,rfc_pred))

              precision    recall  f1-score   support

           1       1.00      0.00      0.00       495
           2       0.00      0.00      0.00       822
           3       0.00      0.00      0.00      1706
           4       0.12      0.00      0.00      2738
           5       0.38      0.01      0.01     15342
           6       0.15      0.00      0.00     11155
           7       0.25      0.05      0.08     23011
           8       0.24      0.95      0.38     31185
           9       0.16      0.00      0.00     20122
          10       0.34      0.02      0.04     23526

    accuracy                           0.24    130102
   macro avg       0.27      0.10      0.05    130102
weighted avg       0.25      0.24      0.12    130102



In [13]:
print(accuracy_score(y_test,rfc_pred)*100)

24.243286037109346
